In [17]:
# Step 1: Install required packages
!pip install -q insightface opencv-python-headless onnxruntime
!pip install -q matplotlib

In [18]:
# Step 2: Import packages
import cv2
import os
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from google.colab.patches import cv2_imshow
from google.colab import files
import shutil

In [21]:
!nvidia-smi

Sun May 25 00:18:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
# Step 3: Prepare directories
known_faces_path = '/content/drive/MyDrive/wanted list/annotation'
output_path = 'output'
os.makedirs(known_faces_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

# Step 4: Load InsightFace face analyzer (retinaface + ArcFace)
face_app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
face_app.prepare(ctx_id=0)

# Step 5: Load known faces
known_embeddings = []
known_names = []

def load_known_faces():
    for file in os.listdir(known_faces_path):
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            path = os.path.join(known_faces_path, file)
            img = cv2.imread(path)
            faces = face_app.get(img)
            if faces:
                known_embeddings.append(faces[0].embedding)
                name = os.path.splitext(file)[0]
                known_names.append(name)
            else:
                print(f"[!] No face detected in: {file}")

load_known_faces()
print(f"✅ Loaded {len(known_names)} known faces.")

# Step 6: Face recognition helper
def recognize(face, threshold=220):
    """
    Compare detected face against known faces using cosine similarity.
    Only return a name if similarity is above a trusted threshold.
    """
    if not known_embeddings:
        return "Unknown", 0.0

    sims = np.dot(known_embeddings, face.embedding)
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]

    if best_score >= threshold:
        return known_names[best_idx], best_score
    return "Unknown", best_score

# Step 7: Process video (webcam or uploaded)
def process_video(video_path=None):
    if video_path:
        cap = cv2.VideoCapture(video_path)
    else:
        cap = cv2.VideoCapture(0)  # Webcam

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_path = os.path.join(output_path, 'annotated_output.avi')
    out = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        faces = face_app.get(frame)

        for face in faces:
            # Get recognition result
            name, best_score = recognize(face, threshold=220)

            box = face.bbox.astype(int)
            if name != "Unknown":
                # Draw green box and name for high-confidence matches
                label = f"{name} ({best_score:.2f})"
                color = (0, 255, 0)  # Green for known faces
            else:
                # Draw red box and "Unknown" for low-confidence matches
                label = f"Unknown ({best_score:.2f})"
                color = (0, 0, 255)  # Red for unknown faces

            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)
            cv2.putText(frame, label, (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        if out is None:
            h, w = frame.shape[:2]
            out = cv2.VideoWriter(out_path, fourcc, 20.0, (w, h))

        out.write(frame)
        cv2_imshow(frame)  # Show frame in Colab

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("✅ Done processing. Saved to:", out_path)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Loaded 17 

In [ ]:
# Step 8: Upload and run
print("📤 Please upload a video file or use the webcam.")

# Uncomment to upload a video:
# uploaded = files.upload()
# video_path = list(uploaded.keys())[0]
process_video("/content/videoooooooooo.mp4")

# Or run from webcam (if running locally):
# process_video()